<a href="https://colab.research.google.com/github/lonespear/MA206/blob/main/colin_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
res_df <- read_csv("results.csv")
driver_df <- read_csv("driver_standings.csv")
const_df <- read_csv("constructor_results.csv")
drivers <- read_csv("drivers.csv") %>% select(driverId, driverRef)
constructors <- read_csv("constructors.csv") %>% select(constructorId, name)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 26759 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): position, positionText, time, milliseconds, fastestLap, rank, fast...
dbl (10): resultId, raceId, driverId, constructorId, number, grid, positionO...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 34863 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (7): driverStandingsId, raceId, driverId, points, position, positionText...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the c

In [7]:
df <- res_df %>%
  left_join(const_df, by = c("raceId", "constructorId")) %>%  # safer join
  left_join(driver_df, by = c("raceId", "driverId")) %>%
  left_join(drivers, by = "driverId") %>%
  left_join(constructors, by = "constructorId")



In [32]:
df %>% count(name, sort=TRUE)

name,n
<chr>,<int>
Mercedes,316
McLaren,315
Ferrari,222
Red Bull,186
Benetton,68
Team Lotus,48
Renault,46
Toro Rosso,23
Maserati,21


In [49]:
df <- df %>% filter((driverId == 830 | driverId == 579 | driverId == 102 |
  driverId == 30 | driverId == 1 | driverId == 117 ) & (name == "Mercedes" | name == "McLaren" | name == "Ferrari" | name == "Red Bull" | name == "Benetton"))

In [54]:
df %>% count(name)

name,n
<chr>,<int>
Benetton,68
Ferrari,222
McLaren,315
Mercedes,316


In [51]:
df %>% colnames

[1] "resultId"             "raceId"               "driverId"            
 [4] "constructorId"        "number"               "grid"                
 [7] "position.x"           "positionText.x"       "positionOrder"       
[10] "points.x"             "laps"                 "time"                
[13] "milliseconds"         "fastestLap"           "rank"                
[16] "fastestLapTime"       "fastestLapSpeed"      "statusId"            
[19] "constructorResultsId" "points.y"             "status"              
[22] "driverStandingsId"    "points"               "position.y"          
[25] "positionText.y"       "wins"                 "driverRef"           
[28] "name"

Points, driver, constructor, grid, position
Is Max Verstappen placing in a higher position per race than other world champtionship winnters?

\\

$$
\widehat{position} = \beta_0 + \beta_1 Driver + \beta_2 Constructor + \beta_3 Grid + \beta_4 Wins
$$

In [52]:
model1 <- lm(as.integer(position.x) ~ driverRef + name + grid + wins , data=df)

Warning message in eval(predvars, data, env):
“NAs introduced by coercion”


In [53]:
summary(model1)


Call:
lm(formula = as.integer(position.x) ~ driverRef + name + grid + 
    wins, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.9987 -1.5130 -0.6104  0.6637 16.7108 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  0.13424    0.77568   0.173  0.86265    
driverRefhamilton           -0.03641    0.65747  -0.055  0.95585    
driverRefmichael_schumacher  1.80549    0.66959   2.696  0.00717 ** 
driverRefprost              -0.68825    0.71631  -0.961  0.33695    
driverRefsenna              -0.15813    0.76568  -0.207  0.83643    
nameFerrari                  0.72243    0.44822   1.612  0.10743    
nameMcLaren                  3.08911    0.58255   5.303 1.50e-07 ***
nameMercedes                 2.86029    0.53778   5.319 1.38e-07 ***
grid                         0.29994    0.02500  11.997  < 2e-16 ***
wins                        -0.22529    0.03595  -6.266 6.24e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 

In [58]:
model2 <- lm(as.integer(position.x) ~ driverRef + name + grid + wins + as.integer(rank), data=df)
summary(model2)

Warning message in eval(predvars, data, env):
“NAs introduced by coercion”
Warning message in eval(predvars, data, env):
“NAs introduced by coercion”



Call:
lm(formula = as.integer(position.x) ~ driverRef + name + grid + 
    wins + as.integer(rank), data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.8969 -1.3662 -0.5363  0.8490 13.3176 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -0.61624    0.66560  -0.926 0.355073    
driverRefmichael_schumacher  1.92744    0.49272   3.912 0.000107 ***
nameMcLaren                  2.39378    0.72187   3.316 0.000994 ***
nameMercedes                 1.97195    0.61075   3.229 0.001344 ** 
grid                         0.19272    0.02873   6.707 6.61e-11 ***
wins                        -0.12415    0.04421  -2.809 0.005215 ** 
as.integer(rank)             0.40484    0.03933  10.293  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.598 on 409 degrees of freedom
  (505 observations deleted due to missingness)
Multiple R-squared:  0.5127,	Adjusted R-squared:  0.50

In [55]:
df$rank[1:5]

[1] "2"  "3"  "19" "3"  "2"